<h1> Equity IQ </h1>
An AI-powered multi-agent financial system using CrewAI/AutoGen and LlamaIndex to automate financial data extraction, analysis, and summarization, enabling more efficient and informed investment decisions.


1. Installing all dependencies

In [ ]:
!pip install crewai
!pip install langchain_groq
!pip install crewai_tools
!pip install langchain_huggingface

!pip install llama_index.llms.groq
!pip install llama-index-embeddings-huggingface
!pip install llama-parse
!pip install llama-index-llms-langchain
!pip install llama_index.evaluation
!pip install ragas

  Using cached langchain_huggingface-0.1.2-py3-none-any.whl.metadata (1.3 kB)
  Using cached tokenizers-0.21.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12

In [ ]:
import os
import json
import csv


from crewai import Agent, Task, Crew

from langchain_groq import ChatGroq

For this project, we are using the Llama 3.3 70B versatile model. This model supports tool calling and is the right fit between performance and compute. We are using Groq APIs for this model. <br/><br/>
We define a global base_llm model

In [ ]:
from google.colab import userdata

os.environ["GROQ_API_KEY"] = "" #use your own api key here

base_llm = ChatGroq(
    model_name="groq/llama-3.3-70b-versatile",
    temperature=0.0,
    max_tokens=2000
)


In [ ]:
import os
os.makedirs('data', exist_ok=True)


!wget "https://s23.q4cdn.com/407969754/files/doc_financials/2019/ar/Uber-Technologies-Inc-2019-Annual-Report.pdf" -O data/uber_10k.pdf
!wget "https://s2.q4cdn.com/470004039/files/doc_earnings/2023/q4/filing/_10-K-Q4-2023-As-Filed.pdf" -O data/apple_2023.pdf
!wget "https://s2.q4cdn.com/470004039/files/doc_financials/2022/q4/_10-K-2022-(As-Filed).pdf" -O data/apple_2022.pdf
!wget "https://s2.q4cdn.com/470004039/files/doc_financials/2021/q4/_10-K-2021-(As-Filed).pdf" -O data/apple_2021.pdf
!wget "https://s2.q4cdn.com/470004039/files/doc_financials/2020/ar/_10-K-2020-(As-Filed).pdf" -O data/apple_2020.pdf
!wget "https://www.dropbox.com/scl/fi/i6vk884ggtq382mu3whfz/apple_2019_10k.pdf?rlkey=eudxh3muxh7kop43ov4bgaj5i&dl=1" -O data/apple_2019.pdf

# download Tesla
!wget "https://ir.tesla.com/_flysystem/s3/sec/000162828024002390/tsla-20231231-gen.pdf" -O data/tesla_2023.pdf
!wget "https://ir.tesla.com/_flysystem/s3/sec/000095017023001409/tsla-20221231-gen.pdf" -O data/tesla_2022.pdf
!wget "https://www.dropbox.com/scl/fi/ptk83fmye7lqr7pz9r6dm/tesla_2021_10k.pdf?rlkey=24kxixeajbw9nru1sd6tg3bye&dl=1" -O data/tesla_2021.pdf
!wget "https://ir.tesla.com/_flysystem/s3/sec/000156459021004599/tsla-10k_20201231-gen.pdf" -O data/tesla_2020.pdf
!wget "https://ir.tesla.com/_flysystem/s3/sec/000156459020004475/tsla-10k_20191231-gen_0.pdf" -O data/tesla_2019.pdf


--2025-04-18 00:50:16--  https://s2.q4cdn.com/470004039/files/doc_earnings/2023/q4/filing/_10-K-Q4-2023-As-Filed.pdf
Resolving s2.q4cdn.com (s2.q4cdn.com)... 68.70.205.3, 68.70.205.2, 68.70.205.4, ...
Connecting to s2.q4cdn.com (s2.q4cdn.com)|68.70.205.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 714094 (697K) [application/pdf]
Saving to: ‘data/apple_2023.pdf’

data/apple_2023.pdf 100%[===================>] 697.36K  --.-KB/s    in 0.04s   

2025-04-18 00:50:16 (18.9 MB/s) - ‘data/apple_2023.pdf’ saved [714094/714094]

--2025-04-18 00:50:16--  https://ir.tesla.com/_flysystem/s3/sec/000162828024002390/tsla-20231231-gen.pdf
Resolving ir.tesla.com (ir.tesla.com)... 23.50.228.30, 2600:1408:9000:684::700, 2600:1408:9000:695::700
Connecting to ir.tesla.com (ir.tesla.com)|23.50.228.30|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/pdf]
Saving to: ‘data/tesla_2023.pdf’

data/tesla_2023.pdf     [ <=>              

<h2> RAG Agent </h2>
The first step in our pipeline is the RAG Agent. Based on the user's selection, the appropriate report is ingested.
<h3> Model </h3>

The RAG Agent uses the Llama3-8b model with a temperature set to 0.0 to promote determinism.

<h3> Embeddings </h3>
The RAG Agent uses BAAI's bge-small-en-v1.5 embeddings for document retrieval.

In [ ]:
!rm -rf vector_store
!pip install llama-index-readers-file

  Using cached llama_index_core-0.12.31-py3-none-any.whl.metadata (2.6 kB)
  Using cached banks-2.1.1-py3-none-any.whl.metadata (11 kB)
  Using cached dirtyjson-1.0.8-py3-none-any.whl.metadata (11 kB)
  Using cached filetype-1.2.0-py2.py3-none-any.whl.metadata (6.5 kB)
  Using cached griffe-1.7.2-py3-none-any.whl.metadata (5.0 kB)
  Using cached colorama-0.4.6-py2.py3-none-any.whl.metadata (17 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 808.7/808.7 kB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.2/129.2 kB 11.1 MB/s eta 0:00:00


In [ ]:
from crewai_tools import PDFSearchTool
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from langchain_openai import ChatOpenAI
from llama_index.core import StorageContext, load_index_from_storage

import os

chat_llm = ChatOpenAI(
    openai_api_base="https://api.groq.com/openai/v1",
    openai_api_key=os.environ['GROQ_API_KEY'],
    model="gemma2-9b-it",
    temperature=0,
)

VECTOR_STORE_DIR = "vector_store"
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

def createVectorStore():

  # loads BAAI/bge-small-en-v1.5

  if os.path.exists(VECTOR_STORE_DIR):
    # rebuild storage context
    storage_context = StorageContext.from_defaults(persist_dir=VECTOR_STORE_DIR)

    # load index
    index = load_index_from_storage(storage_context,embed_model=embed_model)
    print("Vector store loaded successfully!")

  else:
    # get the data to be ingested
    documents = SimpleDirectoryReader("data").load_data()

    # creates vector store index for the documents using the embedding model provided
    index = VectorStoreIndex.from_documents(documents, embed_model=embed_model)

    # create vector store dir if it didn't exist
    os.makedirs(VECTOR_STORE_DIR, exist_ok=True)

    index.storage_context.persist(persist_dir=VECTOR_STORE_DIR)
    print("Vector store created successfully!")
    print("Vector store saved to:", VECTOR_STORE_DIR)


  # create query engine to query the vector store index and generate the context response
  query_engine = index.as_query_engine(similarity_top_k=5,llm=chat_llm, groq_api_key = os.environ['GROQ_API_KEY'])

  return query_engine


vector_store = createVectorStore()


Vector store loaded successfully!


### Querying vector store returns top k semantically match nodes, with the final response from the llm using those context

In [ ]:
vector_store.query("What are the risk factors associated with Tesla?").response

/usr/local/lib/python3.11/dist-packages/llama_index/llms/langchain/utils.py:51: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  for req_key in LC_MessageClass.schema().get("required"):
/usr/local/lib/python3.11/dist-packages/llama_index/llms/langchain/utils.py:51: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  for req_key in LC_MessageClass.schema().get("required"):


"Tesla's success depends on consumer demand for electric vehicles, which is influenced by factors like public perception of electric vehicles, charging infrastructure availability, and competition from other vehicle types.  \n\nThe automotive industry is known for its cyclical nature and volatility, and Tesla's sales could be affected by economic downturns or shifts in consumer preferences.  \n\nAdditionally, Tesla's reliance on lithium-ion batteries and raw materials like lithium, nickel, and cobalt exposes them to price fluctuations and supply chain disruptions. \n\n\nTesla also operates in a highly competitive market and faces risks related to government regulations, economic incentives, and consumer concerns about the company's future viability. \n"

## Evaluate RAG Database

In [ ]:
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import (
    Faithfulness,
    ResponseRelevancy,
    LLMContextPrecisionWithReference,
    LLMContextRecall,
)
from langchain_openai import ChatOpenAI
from ragas.llms import LangchainLLMWrapper
import os
from google.colab import userdata
# # Set up environment variables for Groq
# os.environ["OPENAI_API_KEY"] = userdata.get('GROQ_API_KEY')
# os.environ["OPENAI_BASE_URL"] = "https://api.groq.com/openai/v1"

# Initialize Groq LLM via LangChain
chat_llm = ChatOpenAI(
    openai_api_base="https://api.groq.com/openai/v1",
    openai_api_key=os.environ["GROQ_API_KEY"],
    model="llama3-70b-8192",  # corrected model name if needed
    temperature=0
)

# Wrap it for RAGAS
evaluator_llm = LangchainLLMWrapper(chat_llm)

# # Prepare your evaluation dataset
data = {
    "question": [
        "How many countries was Uber operating in as of 2019?",
        "What is Uber's headquarters address?",
        "What was the gross bookings value in 2019?",
        "How many trips were completed by Uber in 2019?",
        "What is Uber's main source of revenue?"
    ],
    "retrieved_contexts": [
        [
            "Uber was operating in 69 countries as of 2019.",
            "They were present in over 10,000 cities."
        ],
        [
            "Uber's principal executive offices are located at 1455 Market Street, 4th Floor, San Francisco, California 94103."
        ],
        [
            "In 2019, Uber reported $65 billion in gross bookings.",
            "This includes gross bookings across rides, eats, and freight."
        ],
        [
            "Uber completed 7 billion trips in 2019.",
            "This figure includes both ride-sharing and food delivery trips."
        ],
        [
            "Uber's main source of revenue is its ridesharing business.",
            "This includes connecting drivers with passengers for transportation services."
        ]
    ],
    "response": [
        "Uber operated in 69 countries in 2019.",
        "Uber's headquarters is at 1455 Market Street, 4th Floor, San Francisco, CA.",
        "$65 billion",
        "7 billion trips",
        "Uber's main revenue comes from ridesharing services."
    ],
    "ground_truth": [
        "69 countries",
        "1455 Market Street, 4th Floor, San Francisco, California 94103",
        "$65 billion",
        "7 billion",
        "Ridesharing business"
    ]
}


# Create a HuggingFace Dataset
dataset = Dataset.from_dict(data)

# Evaluate using RAGAS
results = evaluate(
    dataset=dataset,
    metrics=[
        Faithfulness(llm=evaluator_llm),
        LLMContextPrecisionWithReference(llm=evaluator_llm),
        LLMContextRecall(llm=evaluator_llm),
    ]
)

# Print results
print(results)
print(results.to_pandas())

# Save to CSV
results.to_pandas().to_csv("groq_llama3_ragas_eval.csv", index=False)


Evaluating:   0%|          | 0/15 [00:00<?, ?it/s]

{'faithfulness': 1.0000, 'llm_context_precision_with_reference': 1.0000, 'context_recall': 1.0000}
                                          user_input  \
0  How many countries was Uber operating in as of...   
1               What is Uber's headquarters address?   
2         What was the gross bookings value in 2019?   
3     How many trips were completed by Uber in 2019?   
4             What is Uber's main source of revenue?   

                                  retrieved_contexts  \
0  [Uber was operating in 69 countries as of 2019...   
1  [Uber's principal executive offices are locate...   
2  [In 2019, Uber reported $65 billion in gross b...   
3  [Uber completed 7 billion trips in 2019., This...   
4  [Uber's main source of revenue is its rideshar...   

                                            response  \
0             Uber operated in 69 countries in 2019.   
1  Uber's headquarters is at 1455 Market Street, ...   
2                                        $65 billion   
3  

# Creating RAG Tool to be used by our Agent

In [ ]:
from crewai_tools import LlamaIndexTool
rag_tool = LlamaIndexTool.from_query_engine(
        vector_store,
        name="FinancialDocumentSearch",
        description="Use this tool to search and extract information from financial documents of companies",
    )
#
rag_tool.args_schema.schema()



<ipython-input-8-416c8e786f90>:8: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  rag_tool.args_schema.schema()


{'description': 'Schema for query tool.',
 'properties': {'query': {'description': 'Search query for the query tool.',
   'title': 'Query',
   'type': 'string'}},
 'required': ['query'],
 'title': 'QueryToolSchema',
 'type': 'object'}

Testing out our RAG Agent

In [ ]:
rag_tool.run("What are the risk factors associated with Tesla ?")

Using Tool: FinancialDocumentSearch


/usr/local/lib/python3.11/dist-packages/llama_index/llms/langchain/utils.py:51: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  for req_key in LC_MessageClass.schema().get("required"):
/usr/local/lib/python3.11/dist-packages/llama_index/llms/langchain/utils.py:51: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  for req_key in LC_MessageClass.schema().get("required"):


ToolOutput(content="Tesla's success depends on consumer demand for electric vehicles, which is influenced by factors like public perception of electric vehicles, charging infrastructure availability, and competition from other vehicle types.  \n\nThe automotive industry is known for its cyclical nature and volatility, and Tesla's sales could be affected by economic downturns or shifts in consumer preferences.  \n\nAdditionally, Tesla relies on a complex supply chain for raw materials like lithium, nickel, and cobalt, which are subject to price fluctuations and supply disruptions.  \n\nGovernment regulations, economic incentives, and global trends also play a role in shaping the electric vehicle market and could impact Tesla's growth. \n\n\n", tool_name='FinancialDocumentSearch', raw_input={'input': 'What are the risk factors associated with Tesla ?'}, raw_output=Response(response="Tesla's success depends on consumer demand for electric vehicles, which is influenced by factors like publ

We use CrewAI's PDF Search tool to create this RAG Agent i.e. we give our RAG Agent the rag_tool we built using crewAI's PDF Search Tool

In [ ]:
rag_agent = Agent(
        role="Financial Data Extractor",
        goal="Extract accurate information from financial documents to answer queries",
        backstory=(
            "You are an expert in extracting relevant financial information from company reports. "
            "You understand complex financial terminology and can identify key metrics and insights."
        ),
        llm=base_llm,
        verbose=True,
        allow_delegation=False,
    )

rag_task = Task(
    description=("Extract information for the question: {question} using the financial document search tool."
    ),
    expected_output="A detailed, accurate response based on the information in the financial documents.",
    agent=rag_agent,
    tools=[rag_tool],
)



<h2> Writer Agent </h2>
This agent is responsible for generating reports from the 10-K filings.

<h3> Model </h3>
The Writer Agent uses the same LLM as BaseLLM but we increase its temperature a little to add some natural variation.

In [ ]:
from transformers import BartForConditionalGeneration, AutoTokenizer
import torch
class SummarizerAgent:
    def __init__(self, model_dir="yatharth97/BART-10K-summarization"):
        """
        Initialize the summarizer agent with the specified BART model.

        Args:
            model_dir (str): The directory or HuggingFace model name for the BART model
        """
        self.model_dir = model_dir
        self.tokenizer = None
        self.model = None
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self._load_model()

    def _load_model(self):
        """Load the BART model and tokenizer"""
        print(f"Loading model from {self.model_dir} to {self.device}...")
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_dir)
        self.model = BartForConditionalGeneration.from_pretrained(self.model_dir)
        self.model = self.model.to(self.device)
        print("Model loaded successfully.")

    def summarize_text(self, text, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True):
        """
        Generate a summary for the input text.

        Args:
            text (str): The text to summarize
            max_length (int): Maximum length of the summary
            min_length (int): Minimum length of the summary
            length_penalty (float): Exponential penalty to the length
            num_beams (int): Number of beams for beam search
            early_stopping (bool): Whether to stop beam search when finished

        Returns:
            str: The generated summary
        """
        if not text:
            return "No text provided for summarization."

        # Encode the text
        inputs = self.tokenizer(text, max_length=1024, return_tensors="pt", truncation=True, padding="max_length")

        # Move input tensors to the same device as the model
        input_ids = inputs.input_ids.to(self.device)
        attention_mask = inputs.attention_mask.to(self.device)

        # Generate summary
        summary_ids = self.model.generate(
            input_ids,
            attention_mask=attention_mask,
            max_length=max_length,
            min_length=min_length,
            length_penalty=length_penalty,
            num_beams=num_beams,
            early_stopping=early_stopping
        )

        # Decode the generated ids to text
        summary = self.tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        print("Summary generated by BART: ", summary)
        return summary

summarizer = SummarizerAgent()
# summarizer.summarize_text(text_to_summarize). #summarize the text like this


from crewai.tools import tool

@tool
def generate_summary_with_bart(text: str, max_length: int = 150) -> str:
    """
    Generate a summary of the provided text using the BART model.

    Args:
        text (str): Text to summarize
        max_length (int): Maximum length of summary

    Returns:
        str: Generated summary
    """
    return summarizer.summarize_text("Summarize:" + text, max_length=max_length)


Loading model from yatharth97/BART-10K-summarization to cpu...
Model loaded successfully.


In [ ]:
writing_llm = ChatGroq(
    model_name="groq/llama-3.3-70b-versatile",
    temperature=0.7,
    max_tokens=2000
)

writer_agent = Agent(
        role="Financial Report Writer",
        goal="Compose concise, informative summaries of financial data",
        backstory=(
            "You are a skilled financial writer who can translate complex financial data into "
            "clear, actionable insights. You know how to highlight the most important information "
            "and present it in a structured, readable format."
        ),
        llm=writing_llm,
        verbose=True,
        allow_delegation=False,
    )

summarizer_agent = Agent(
        role="Content Summarizer",
        goal="Generate concise and accurate summaries of provided content",
        backstory="I am an AI agent specialized in summarizing text using a fine-tuned BART model. I can process various types of content and create summaries that capture the essential points.",
        verbose=True,
        llm=writing_llm,
        tools=[generate_summary_with_bart]
    )

# writer_task = Task(
#         description=(
#             "Based on the extracted information, create a well-structured financial summary that answers the question. "
#             "Focus on clarity, accuracy, and presenting the information in a useful format."
#         ),
#         expected_output="A clear, concise, and informative financial summary answering the user's question.",
#         agent=writer_agent,
#         context=[rag_task],
#     )

writer_task = Task(
    description=(
        "Create a financial report that directly answers the user's question based on the information retrieved by the RAG system. "
        "The report should be clear, focused, and use only the information provided by the RAG task."
    ),
    expected_output="A financial report that precisely answers the user's question using the retrieved information.",
    agent=writer_agent,
    context=[rag_task],
)

summarizer_task= Task(
        description="Summarize the provided content accurately and concisely",
        agent=summarizer_agent,
        expected_output="A concise summary of the provided content",
        context=[writer_task],
    )

We now have the first part of our RAG Agent - Writer Agent pipeline setup! We can formally define this pipeline in CrewAI

In [ ]:
from crewai import Process
financial_crew = Crew(
            agents=[rag_agent, writer_agent],
            tasks=[rag_task, writer_task],
            verbose=True,
            process=Process.sequential,  # Run tasks in sequence
        )

In [ ]:
inputs = {"question":"What are the risk factors associated with Tesla ?"}
result = financial_crew.kickoff(inputs=inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 953e7c80-fd5e-47cf-a965-d0b419daffac                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: c01e68c2-b1da-4201-a5e4-740ad42aba2a
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: c01e68c2-b1da-4201-a5e4-740ad42aba2a
       Status: Executing Task...
    └── 🤖 Agent: Financial Data Extractor
            Status: In Progress

# Agent: Financial Data Extractor
## Task: Extract information for the question: What are the risk factors associated with Tesla ? using the financial document search tool.


🤖 Agent: Financial Data Extractor
    Status: In Progress

...

# Agent: Financial Report Writer
## Task: Create a financial report that directly answers the user's question based on the information retrieved by the RAG system. The report should be clear, focused, and use only the information provided by the RAG task.


# Agent: Financial Report Writer
## Final Answer: 
**Tesla Financial Report**

**Introduction:**
This report provides an overview of the key risks and challenges facing Tesla, Inc., a leading electric vehicle manufacturer. The report highlights the potential impact of these risks on Tesla's financial performance and provides insights into the company's ability to mitigate these risks.

**Risk Factors:**

1. **Production and Launch Delays:**
Tesla faces significant risks related to production and launch delays, which can impact the company's ability to meet consumer demand and achieve revenue targets. Delays in launching new products and features can result in lost sales, reduced market share, and decreased profitability. To mitigate

🚀 Crew: crew
├── 📋 Task: c01e68c2-b1da-4201-a5e4-740ad42aba2a
│      Assigned to: Financial Data Extractor
│      Status: ✅ Completed
│   └── 🤖 Agent: Financial Data Extractor
│           Status: ✅ Completed
└── 📋 Task: 987936df-a51b-4a7c-a4a6-a16b1dc9ea4a
       Status: Executing Task...
    └── 🤖 Agent: Financial Report Writer
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: c01e68c2-b1da-4201-a5e4-740ad42aba2a
│      Assigned to: Financial Data Extractor
│      Status: ✅ Completed
│   └── 🤖 Agent: Financial Data Extractor
│           Status: ✅ Completed
└── 📋 Task: 987936df-a51b-4a7c-a4a6-a16b1dc9ea4a
       Assigned to: Financial Report Writer
       Status: ✅ Completed
    └── 🤖 Agent: Financial Report Writer
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 987936df-a51b-4a7c-a4a6-a16b1dc9ea4a                                                                     │
│  Agent: Financial Report Writer                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 953e7c80-fd5e-47cf-a965-d0b419daffac                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
from IPython.display import Markdown, display

print(display(Markdown(result.raw)))
# print(writer_task.output.raw)
# print(summarizer_task.output.raw)

**Tesla Financial Report**

**Introduction:**
This report provides an overview of the key risks and challenges facing Tesla, Inc., a leading electric vehicle manufacturer. The report highlights the potential impact of these risks on Tesla's financial performance and provides insights into the company's ability to mitigate these risks.

**Risk Factors:**

1. **Production and Launch Delays:**
Tesla faces significant risks related to production and launch delays, which can impact the company's ability to meet consumer demand and achieve revenue targets. Delays in launching new products and features can result in lost sales, reduced market share, and decreased profitability. To mitigate this risk, Tesla must ensure that its manufacturing processes are efficient, and its supply chain is reliable.

2. **Battery Cell Supply and Costs:**
The cost and availability of battery cells are critical components of Tesla's products. Fluctuations in raw material prices, supply chain disruptions, and supplier reliability issues can impact Tesla's profitability and growth. The company must closely monitor its battery cell supply chain and develop strategies to mitigate potential disruptions.

3. **Market Demand and Competition:**
The success of Tesla depends on consumer demand for electric vehicles, which is subject to various factors, including consumer perceptions, government regulations, and the overall economic climate. The market for electric vehicles is highly competitive, with established and new automakers entering the market. Tesla must continue to innovate and improve its products to maintain its market share and competitiveness.

4. **Interest Rate Sensitivity:**
Rising interest rates may lead to reduced consumer spending, potentially impacting demand for Tesla's products. The company must monitor interest rate changes and adjust its pricing and financing strategies accordingly to mitigate the impact of higher interest rates.

5. **Strong Competition:**
Tesla faces strong competition from established and new automakers entering the electric vehicle market. These competitors may have more resources and established market presence, posing a challenge to Tesla's market share and profitability. To remain competitive, Tesla must continue to invest in research and development, improve its manufacturing efficiency, and enhance its brand reputation.

**Conclusion:**
Tesla faces significant risks and challenges that can impact its financial performance. However, the company has a strong track record of innovation and has demonstrated its ability to adapt to changing market conditions. By monitoring and mitigating these risks, Tesla can maintain its position as a leader in the electric vehicle market and achieve long-term growth and profitability.

**Recommendations:**

* Tesla should continue to invest in research and development to improve its products and stay ahead of the competition.
* The company should closely monitor its supply chain and develop strategies to mitigate potential disruptions.
* Tesla should maintain a strong brand reputation and focus on customer satisfaction to maintain its market share.
* The company should monitor interest rate changes and adjust its pricing and financing strategies accordingly.
* Tesla should continue to expand its product offerings and enter new markets to diversify its revenue streams and reduce its dependence on a single market.

By following these recommendations, Tesla can minimize the impact of these risks and achieve long-term success in the electric vehicle market.

None


<h1> Briefing Agent </h1>

In [ ]:
from transformers import BartForConditionalGeneration, AutoTokenizer
import torch

model_dir = "/data" # provide our own model path

tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = BartForConditionalGeneration.from_pretrained(model_dir)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
def generate_summary(text, max_length=250, min_length=40, length_penalty=2.0, num_beams=4):
    """
    Generate a summary for the input text.

    Args:
        text (str): The text to summarize
        max_length (int): Maximum length of the summary
        min_length (int): Minimum length of the summary
        length_penalty (float): Exponential penalty to the length. Values < 1.0 encourage shorter sequences,
                               values > 1.0 encourage longer sequences
        num_beams (int): Number of beams for beam search. 1 means no beam search
        early_stopping (bool): Whether to stop beam search when at least `num_beams` sentences are finished per batch

    Returns:
        str: The generated summary
    """
    # Encode the text
    inputs = tokenizer(text, max_length=1024, return_tensors="pt", truncation=True, padding="max_length")

    # Move input tensors to the same device as the model
    input_ids = inputs.input_ids.to(device)
    attention_mask = inputs.attention_mask.to(device)

    # Generate summary
    summary_ids = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_length=max_length,
        min_length=min_length,
        length_penalty=length_penalty,
        num_beams=num_beams,
        # early_stopping=early_stopping
    )

    # Decode the generated ids to text
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    return summary

# Example usage
text_to_summarize = """
Summarize:

Financial Report: Assessment of Risks and Challenges Facing Tesla

Introduction:
This financial report provides an analysis of the key risks and challenges facing Tesla, Inc., a leading manufacturer of electric vehicles. The report is based on the information retrieved by the RAG system and focuses on the potential impact of these risks on the company's financial performance and growth prospects.

Production and Launch Delays:
Tesla faces significant risks related to production and launch delays, which can result in lost sales, reputational damage, and increased costs. Delays in launching new products and features can lead to a loss of market share and revenue, as competitors may fill the gap with their own offerings. The company's ability to manage its supply chain, manufacturing processes, and new product introductions is critical to mitigating this risk.

Battery Cell Supply and Costs:
The availability and cost of battery cells are crucial to Tesla's operations, as they are a key component of electric vehicles. Fluctuations in raw material prices, supply chain disruptions, and supplier reliability issues can impact the company's profitability and growth. Tesla's reliance on battery cells manufactured by itself or suppliers exposes it to risks related to material costs, availability, and supplier reliability.

Market Demand and Competition:
The success of Tesla depends on consumer demand for electric vehicles, which is influenced by various factors, including consumer perceptions, government regulations, and the overall economic climate. The market for electric vehicles is highly competitive, with established and new automakers entering the market. Tesla faces strong competition from companies with more resources and established market presence, posing a challenge to its market share and profitability.

Interest Rate Sensitivity:
Rising interest rates may lead to reduced consumer spending, potentially impacting demand for Tesla's products. Higher interest rates can increase the cost of borrowing for consumers, making electric vehicles less affordable and reducing demand. This, in turn, can impact Tesla's revenue and profitability.

Strong Competition:
Tesla operates in a highly competitive market, with many established and new automakers offering electric vehicles. The company faces significant competition from companies with more resources, established market presence, and brand recognition. This competition can impact Tesla's market share, revenue, and profitability, as well as its ability to achieve its growth objectives.

Conclusion:
In conclusion, Tesla faces significant risks and challenges that can impact its financial performance and growth prospects. The company's ability to manage production and launch delays, battery cell supply and costs, market demand and competition, interest rate sensitivity, and strong competition will be critical to its success. By understanding and mitigating these risks, Tesla can minimize their potential impact and achieve its growth objectives.

Recommendations:
Based on the analysis of the risks and challenges facing Tesla, we recommend that the company:

1. **Diversify its supply chain:** To reduce the risk of supply chain disruptions and material cost fluctuations, Tesla should consider diversifying its supply chain and exploring alternative sources of battery cells and other critical components.
2. **Invest in manufacturing efficiency:** To mitigate the risk of production and launch delays, Tesla should invest in manufacturing efficiency and process improvements to reduce the time and cost of launching new products and features.
3. **Monitor market trends and competition:** To stay competitive in the electric vehicle market, Tesla should continuously monitor market trends, consumer perceptions, and competitor activity to identify opportunities and threats.
4. **Develop strategic partnerships:** To reduce the risk of competition and improve its market presence, Tesla should consider developing strategic partnerships with other companies to expand its product offerings and reach new markets.

By implementing these recommendations, Tesla can reduce the potential impact of the risks and challenges facing the company and achieve its growth objectives.
"""


summary = generate_summary(text_to_summarize)
print("Summary:", summary)


Summary: **Summary of Risks and Challenges Facing Tesla, Inc.**

**Market Demand and Competition:**
* Battery cell supply disruptions and supplier reliability issues can impact demand and revenue.
* Higher interest rates can reduce consumer spending, increasing costs. **Diversification and production efficiency:** Tesla is in a highly competitive market, with established and new automakers entering the electric vehicle market. The company's success depends on securing batteries and managing supply chain disruptions and material cost fluctuations. To mitigate these risks, Tesla should diversify its supply chain and explore alternative sources of battery cells and other critical components.
**Interest Rate Sensitivity:** The company faces significant competition from companies with resources and established market presence. A rising interest rate would impact demand for electric vehicles less affordable and reducing demand for products. This could negatively impact Tesla's financial perf

In [ ]:
def briefing_pipeline(result):
   input_ids = tokenizer(
        document,
        return_tensors="pt",
        max_length=1024,
        truncation=True,
        padding="max_length"
    ).input_ids.to(model.device)
  output_ids = model.generate(input_ids, max_length=256, num_beams=5)
  generated_summary = tokenizer.decode(output_ids[0], skip_special_tokens=True)

  # Print the generated summary
  print("\nGenerated Summary:")
  print(generated_summary)
  print("\n" + "-"*50 + "\n")



<h1> Analytics Agent </h1>
This agent is designed to automate the extraction and structuring of financial information from SEC filings, such as 10-K reports.

Importing all the required libraries.

In [ ]:
from typing import List
from pydantic import BaseModel
import os
import json
import csv
from tqdm import tqdm


from llama_index.core.llms import ChatMessage
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_parse import LlamaParse
from llama_index.llms.groq import Groq
from llama_index.core.schema import Document
from llama_index.core import Settings
import nest_asyncio

nest_asyncio.apply()

In [ ]:
def update_stock_analysis_csv(json_str: str, csv_path="stock_analysis.csv"):

    try:
        data = json.loads(json_str)
    except json.JSONDecodeError:
        print("Could not parse JSON:", json_str)
        return

    # Fields we want in each row:
    fieldnames = ["company", "year", "revenue", "net_income", "eps", "cash_flow"]

    # Check if the file exists to decide if we need to write a header
    write_header = not os.path.exists(csv_path)

    with open(csv_path, "a", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        if write_header:
            writer.writeheader()

        # Create a row from data
        row = {
            "company": data.get("company", "N/A"),
            "year": data.get("year", "N/A"),
            "revenue": data.get("revenue", "N/A"),
            "net_income": data.get("net_income", "N/A"),
            "eps": data.get("eps", "N/A"),
            "cash_flow": data.get("cash_flow", "N/A"),
        }
        writer.writerow(row)


In [ ]:
def parse_file(file_path: str) -> List[Document]:
    llama_parse = LlamaParse(
        api_key="llx-gifpKW3ErlDpstQxOpJqW6WJDnTEtBAD3OtYXrwpoA1m4pyb",  #TODO: Parameterize LLamacloud API
        result_type='markdown',
    )

    result = llama_parse.load_data(
        file_path,
    )
    return result

We now define the excel file we want the Analytics Agent to fill

In [ ]:
documents = parse_file('data/stock_analysis.xlsx')

Started parsing the file under job_id 122247ee-b1b1-4be8-9364-d7adc884798b


We can see the financial parameters we are asking the Analytics Agent to fill

In [ ]:
print(documents[0].text)

|Parameter|2019|
|---|---|
| |Uber|
|1. Revenue| |
|2. Net Income| |
|3. Earnings Per Share (EPS)| |
|4. EBITDA| |
|5. Free Cash Flow| |
|6. Return on Equity (ROE)| |
|7. Return on Assets (ROA)| |
|8. Debt-to-Equity Ratio| |
|9. Current Ratio| |
|10. Gross Margin| |
|11. Operating Margin| |
|12. Net Profit Margin| |
|13. Inventory Turnover| |
|14. Accounts Receivable Turnover| |
|15. Capital Expenditures| |
|16. Research and Development Expenses| |
|17. Market Cap| |
|18. Price-to-Earnings (P/E) Ratio| |
|19. Dividend Yield| |
|20. Year-over-Year Growth Rate| |



In [ ]:
text = documents[0].text

Prompt Engineering to come up with the optimal prompt for correctly foirmatted results

In [ ]:
prompt = f"""
You are an AI assistant specializing in financial analysis. You've been given an Excel spreadsheet containing financial data for multiple companies. Your task is to extract and structure this information in a clear, organized format.

The Excel sheet contains the following:
1. Multiple companies (rows)
2. Year (columns)
3. Various financial parameters (sub-columns)

Input Excel data:
{text}

Please present the extracted and structured information in a clear, easy-to-read format.
"""

In [ ]:
class CompanyParameters(BaseModel):
    """Data model for an sec filing analysis."""

    Companies: List[str]
    FinancialParameters: List[str]
    Years: List[str]

In [ ]:
llm = Groq(model="llama3-70b-8192", api_key="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")

In [ ]:
from llama_index.core.llms import ChatMessage

sllm = llm.as_structured_llm(output_cls=CompanyParameters)
input_msg = ChatMessage.from_str(prompt)

In [ ]:
output = sllm.chat([input_msg])

In [ ]:
output_obj = output.raw

In [ ]:
output_obj

CompanyParameters(Companies=['Uber'], FinancialParameters=['Revenue', 'Net Income', 'Earnings Per Share (EPS)', 'EBITDA', 'Free Cash Flow', 'Return on Equity (ROE)', 'Return on Assets (ROA)', 'Debt-to-Equity Ratio', 'Current Ratio', 'Gross Margin', 'Operating Margin', 'Net Profit Margin', 'Inventory Turnover', 'Accounts Receivable Turnover', 'Capital Expenditures', 'Research and Development Expenses', 'Market Cap', 'Price-to-Earnings (P/E) Ratio', 'Dividend Yield', 'Year-over-Year Growth Rate'], Years=['2019'])

In [ ]:
companies = ["Uber"]
financial_parameters = output_obj.FinancialParameters
years = output_obj.Years

In [ ]:
query = f"what is the Revenue of Uberfor the year 2019?. Don't be verbose. Provide only 1 word for mathematical value nothing more. If you are unable to provide answer, output as NA."


rag_tool.run(query)

Using Tool: Search a PDF's content


/usr/local/lib/python3.11/dist-packages/chromadb/types.py:144: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x


'Relevant Content:\n'

In [ ]:
analytics_pipeline = Crew(
    agents=[RAG_Agent],
    tasks=[RAG_task],
    verbose=True,
    tpm_limit=1000,
)

In [ ]:
from tqdm import tqdm
import asyncio
async def generate_answers(companies: List[str], financial_parameters: List[str], years: List[str]) -> List[str]:
    companies_financial_parameters_answers = {}

    for year in years:
        companies_financial_parameters_answers[year] = {}
        for company in companies:
            companies_financial_parameters_answers[year][company] = {}
            for financial_parameter in tqdm(financial_parameters):
                query = f"what is the {financial_parameter} of {company} for the year {year}?. Don't be verbose. Provide only 1 word for mathematical value nothing more. If you are unable to provide answer, output as NA."
                answer =  analytics_pipeline.kickoff(inputs={"question": query})

                await asyncio.sleep(60)
                companies_financial_parameters_answers[year][company][financial_parameter] = answer

    return companies_financial_parameters_answers

In [ ]:
# answers =await generate_answers(companies, financial_parameters, years)

In [ ]:
if __name__ == "__main__":


    company_queries = [
        ("Uber", "2019"),
    ]

    for (company, year) in company_queries:
        print("\n\n============================================================")
        print(f"Processing {company} ({year})...")



        inputs = {f"question":"What are the risk factors associated with {company} ?"}
        result = financial_crew.kickoff(inputs=inputs)
        try:
          briefing_pipeline(result)
        except:
          print("Briefing agent is offline!")

        answers =await generate_answers(companies, financial_parameters, years)




Processing Uber (2019)...


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 12fb52c5-3c94-4dcf-80b8-f85145b6d1ed                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 95e4a04d-0505-41a0-a1d5-afe71aed60d9
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 95e4a04d-0505-41a0-a1d5-afe71aed60d9
       Status: Executing Task...
    └── 🤖 Agent: Financial Data Extractor
            Status: In Progress

# Agent: Financial Data Extractor
## Task: Extract information for the question What are the risk factors associated with {company} ? with the help of the respective tool.


...

🚀 Crew: crew
└── 📋 Task: 95e4a04d-0505-41a0-a1d5-afe71aed60d9
       Status: Executing Task...
    └── 🤖 Agent: Financial Data Extractor
            Status: ✅ Completed

🚀 Crew: crew
└── 📋 Task: 95e4a04d-0505-41a0-a1d5-afe71aed60d9
       Assigned to: Financial Data Extractor
       Status: ✅ Completed
    └── 🤖 Agent: Financial Data Extractor
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 95e4a04d-0505-41a0-a1d5-afe71aed60d9                                                                     │
│  Agent: Financial Data Extractor                                                                                │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: e82e852d-8083-4f7b-8c56-ec46dd559578                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

100%|██████████| 20/20 [21:27<00:00, 64.35s/it]


In [ ]:
answers

{'2019': {'Uber': {'Revenue': CrewOutput(raw='14526', pydantic=None, json_dict=None, tasks_output=[TaskOutput(description="Extract information for the question what is the Revenue of Uber for the year 2019?. Don't be verbose. Provide only 1 word for mathematical value nothing more. If you are unable to provide answer, output as NA. with the help of the respective tool.", name=None, expected_output='Return a clear and consise text as response.', summary='Extract information for the question what is the Revenue of...', raw='14526', pydantic=None, json_dict=None, agent='Financial Data Extractor', output_format=<OutputFormat.RAW: 'raw'>)], token_usage=UsageMetrics(total_tokens=5461, prompt_tokens=3097, cached_prompt_tokens=0, completion_tokens=2364, successful_requests=6)),
   'Net Income': CrewOutput(raw='-8.5 billion', pydantic=None, json_dict=None, tasks_output=[TaskOutput(description="Extract information for the question what is the Net Income of Uber for the year 2019?. Don't be verbo

In [ ]:
def flatten_dict(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

# Flatten the nested dictionary
flat_data = []
for year, companies in answers.items():
    for company, metrics in companies.items():
        flat_metrics = flatten_dict(metrics)
        flat_metrics['Year'] = year
        flat_metrics['Company'] = company
        flat_data.append(flat_metrics)

# Get all unique keys to use as CSV headers
headers = set()
for item in flat_data:
    headers.update(item.keys())

# Sort headers to ensure 'Year' and 'Company' come first
headers = sorted(headers)
headers.insert(0, headers.pop(headers.index('Year')))
headers.insert(1, headers.pop(headers.index('Company')))

# Write to CSV
with open('sec_10k_analysis_form_filling.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=headers)
    writer.writeheader()
    for row in flat_data:
        writer.writerow(row)

In [ ]:
import pandas as pd
from IPython.core.display import HTML

pd.set_option('display.max_colwidth', 10)

out_df = pd.read_csv("sec_10k_analysis_form_filling.csv")

In [ ]:
html = out_df.to_html()
HTML(html)

,Year,Company,Accounts Receivable Turnover,Capital Expenditures,Current Ratio,Debt-to-Equity Ratio,Dividend Yield,EBITDA,Earnings Per Share (EPS),Free Cash Flow,Gross Margin,Inventory Turnover,Market Cap,Net Income,Net Profit Margin,Operating Margin,Price-to-Earnings (P/E) Ratio,Research and Development Expenses,Return on Assets (ROA),Return on Equity (ROE),Revenue,Year-over-Year Growth Rate
0,2019,Uber,NaN,2223,1.3,3.45,NaN,4.8B,($6.81),"$2,536,000",43.9,NaN,55.4B,-8.5 billion,NaN,0,NaN,4.5B,4.7%,NaN,14526,NaN
